# TP 6 : types construits

## Gestion des overflow

On considère le type suivant :

In [3]:
type entier = Infini | MoinsInfini | Entier of int

type entier = Infini | MoinsInfini | Entier of int


On rappelle que `max_int` est le plus grand entier représentable en OCaml et `min_int` le plus petit :

In [1]:
max_int;;
min_int;;
max_int + 1;;  (* un dépassement du plus grand entier donne le plus petit *)

- : int = 4611686018427387903


- : int = -4611686018427387904


- : int = -4611686018427387904


On voudrait pouvoir additionner deux `entier` $a$ et $b$ en prennant en compte les dépassements de `int` (integer overflow), et en donnant `Infini` dans ce cas.  
1. Pourquoi ne peut-on pas tout simplement tester si $a + b >$ `max_int` ?   
2. Pourquoi peut-on tester à la place si `max_int` $ - ~a \leq b$ avec $a \geq 0$ ?
3. Quel test similaire pourrait-on utiliser pour savoir si $a + b <$ `min_int` ?
4. Écrire une fonction `add_int : int -> int -> entier` telle que `add_int` ajoute 2 `int`en tenant compte des dépassements (on renvoie `Infini` s'il y a un dépassement de `max_int` et `MoinsInfini` s'il y a un dépassement de `min_int` par le bas).  
5. Écrire une fonction `add : entier -> entier -> entier` ajoutant 2 entiers en tenant compte des dépassements.  
On pourra écrire `failwith "Indeterminé"` dans le cas d'une forme indéterminé.
6. Écrire une fonction `oppose : entier -> entier` renvoyant l'opposé d'un entier. Par exemple, `oppose Infini` renvoie `MoinsInfini`, `oppose (Entier a)` renvoie `Entier (-a)...`
7. Écrire une fonction `sub` effectuant la soustraction.
8. Pour savoir si $a \times b$ fait un dépassement, on propose la façon suivante :
- Calculer $c = a \times b$
- Regarder si $c/a$ est égal à $b$  
Pourquoi cela fonctionne ? À quelle autre erreur faut-il faire attention dans cette méthode ?
9. Écrire une fonction `mult : entier -> entier -> entier` multipliant 2 entiers en tenant compte des dépassements

1. on ne peut pas écrire a+b > max_int puisque si a+b > max_int alors a+b ne sera la vrai valeur de la somme, mais un nombre allant de min_int à max_int, la vérification sera donc fausse.

2. Nous pouvons tester si max_int - a <= b avec a >= 0, puisque si max_int - a est inférieur à b cela veut dire que max_int est inférieur à a+b mais nous éviterons les problèmes de retour à la plus petite valeur min_int.

3. Nous pourrons vérifier si la somme de deux nombres supérieurs à zéro, ne devient pas un nombre négatif. Si ce nombre devient négatif alors a+b aura dépassé max_int.

In [36]:
let add_int a b = 
    if max_int - a <= b then Infini
    else if min_int + a >= b then MoinsInfini
        else Entier(a+b);;

add_int 5 (max_int-1)

val add_int : int -> int -> entier = <fun>


- : entier = Infini


In [9]:
let add a b = match a,b with
    | Entier(a1), Entier(b1) when max_int - a1 <= b1 && a1 > 0 -> Infini
    | Entier(a1), Entier(b1) when min_int + a1 >= b1 && a1 > 0 -> MoinsInfini
    | Infini, Entier(_) | Entier(_), Infini -> Infini
    | Entier(_), MoinsInfini | MoinsInfini, Entier(_) -> MoinsInfini
    | Entier(a1), Entier(b1) -> Entier(a1+b1)
    | _ -> failwith "Indeterminé";;

add (Entier 200) (Entier (max_int -1))

val add : entier -> entier -> entier = <fun>


- : entier = Infini


In [15]:
let oppose a = match a with
    | Entier(a1) when a1 = min_int -> Infini
    | Infini -> MoinsInfini
    | Entier(a1) -> Entier(-a1)
    | MoinsInfini -> Infini;;

oppose (Entier min_int)

val oppose : entier -> entier = <fun>


- : entier = Infini


In [4]:
let sub a b = match a,b with
    | Entier(a1), Entier(b1) -> Entier(a1 - b1)
    | Infini, Entier(b1) | Entier(b1), MoinsInfini -> Infini
    | MoinsInfini, Entier(a1) | Entier(a1), Infini -> MoinsInfini
    | _ -> failwith "Indeterminé";;
    
sub (Entier 5) MoinsInfini

val sub : entier -> entier -> entier = <fun>


- : entier = Infini


In [16]:
let sub_cor a b = 
    add a (oppose b);;

val sub_cor : entier -> entier -> entier = <fun>


8. Cette manipulation fonctionne car c/a = b est une simple résolution d'équation pour trouver la valeur de b. Si nous trouvons pas la bonne valeur, c'est que c a fait un dépassement car c = a*b

In [6]:
let mut a b = match a,b with 
    | Entier(a1), Entier(b1) when Entier((a1*b1)/a1) <> Entier(b1) && ((Entier(a1)>Entier(0) && Entier(b1) > Entier(0))||(Entier(a1)<Entier(0) && Entier(b1) < Entier(0) )) -> Infini
    | Entier(a1), Entier(b1) when Entier((a1*b1)/a1) <> Entier(b1) -> MoinsInfini
    | Entier(a1), Entier(b1) -> Entier (a1*b1)
    | Infini, Infini | MoinsInfini, MoinsInfini -> Infini
    | MoinsInfini, Infini | Infini, MoinsInfini -> MoinsInfini
    | Entier(a1), Infini | Infini, Entier(a1) when a1 < 0 -> if a1 < 0 then MoinsInfini else Infini
    | Entier(a1), Infini | Infini, Entier(a1) when a1 > 0 -> Infini
    | Entier(a1), MoinsInfini | MoinsInfini, Entier(a1) when a1 < 0 -> if a1 < 0 then Infini else if a1 >  0 then MoinsInfini else Entier(0)
    | _ -> failwith "Indeterminé";;
    
mut (Entier max_int) (Entier 5)
    

val mut : entier -> entier -> entier = <fun>


- : entier = Infini


1. on ne peut pas écrire a+b > max_int puisque si a+b > max_int ce nom la vérification sera donc fausse


1. Par définition, aucun entier ne peut être supérieur à `max_int`.
2. Déjà, $a + b >$ `max_int` implique que $a \geq 0$ (car $b$ est un entier dans est inférieur à `max_int`). De plus `max_int` $- a$ n'effectue pas de dépassement. Donc le test fonctionne sans risque de dépassement.
3. $a + b < $`min_int` se réécrit en $a < 0$ et `min_int`$ - a > b$.  

## Tableaux dynamiques

Il est impossible d'ajouter ou supprimer un élément dans tableau (`array`) en OCaml.  
Les tableaux dynamiques (ou : redimensionnables) permettent d'ajouter un élément `e`, en recréant un tableau plus grand dans lequel on recopie tous les éléments ainsi que `e`.  
Voici la définition de tableau dynamique que nous allons utiliser :

In [1]:
type 'a array_dyn = {mutable t : 'a array; mutable n : int};;

type 'a array_dyn = { mutable t : 'a array; mutable n : int; }


`n` indique le nombre de cases du tableau `t` que l'on considère comme faisant partie du tableau dynamique. Les indices au delà de `n` dans `t` sont ignorés.  
À chaque fois que l'on voudra ajouter un élément `e` à un tableau dynamique `d` :
- s'il reste de la place dans `d.t` (c'est à dire si `d.n < Array.length t`), on met `e` dans `e` dans `d.t.(n)` et on met à jour `d.n`
- sinon, on créé un nouveau tableau `t'` de taille `2n`, on recopie `d.t` dedans, on ajoute `e` puis on remplace `d.t` par `t'.`

1. Écrire une fonction `add` ajoutant un élément dans un `array_dyn`. 
2. Quelle est la complexité dans le pire cas de `add` ?
3. On suppose que l'on ajoute $n$ éléments (avec `add` à un tableau dynamique de taille initiale 1. Montrer que la complexité totale de ces $n$ opérations est O($n$) (autrement dit, la complexité moyenne ou **complexité amortie** d'une opération est O(1))

In [18]:
let exemple = {t = [|0|]; n = 0}

val exemple : int array_dyn = {t = [|0|]; n = 0}


In [22]:
let new_tab t = 
    let new_t = Array.make ((Array.length t)*2) 0 in
    for i = 0 to Array.length t -1 do 
        new_t.(i) <- t.(i)
    done;
    new_t;;

let add e d =
    d.n <- (d.n + 1);
    if (d.n < Array.length d.t then d.t.((d.n)-1) <- e
    else d.t <- (new_tab d.t); d.t.((d.n)-1) <- e;;
    

add 5 exemple;;
exemple

val new_tab : int array -> int array = <fun>


val add : int -> int array_dyn -> unit = <fun>


- : unit = ()


- : int array_dyn = {t = [|5; 5; 5; 5; 0; 0; 0; 0|]; n = 4}


2. Quelle est la complexité dans le pire cas de `add` ? 

Ne savant pas la complexité de la fonction Array.make, nous allons considérer que celle-ci est linéaire.

Le seul moment où la compléxité n'est pas de 1 est la boucle for de la fonction new_tab qui donne une compléxité O(n).
La complexité de la fonction est donc O(n).

3. On suppose que l'on ajoute $n$ éléments (avec `add` à un tableau dynamique de taille initiale 1. Montrer que la complexité totale de ces $n$ opérations est O($n$) (autrement dit, la complexité moyenne ou **complexité amortie** d'une opération est O(1))

Lorsqu'on ajoute n éléments, on répété n-fois la fonction add de complexité O(n)